<a href="https://colab.research.google.com/github/Fireflash135/samples/blob/main/SMCP_Models_notebook_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**SMCP Project**

**<center>Predicting future sales of SMCP according to whether the products are new or already existing**

# Setup

In [ ]:
!pip install "snowflake-connector-python[secure-local-storage,pandas]"
!pip install -U pyasn1 pyasn1-modules
!pip install --upgrade snowflake-sqlalchemy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold
from sklearn import linear_model, metrics, preprocessing
from sklearn.dummy import DummyRegressor as dr
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import snowflake.connector
import sqlalchemy
from sqlalchemy import inspect
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge, Lasso
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras import callbacks
from keras.callbacks import ModelCheckpoint

In [ ]:
url = URL(
    account = 'sb10435',
    user = '###', # Input your snowflake username (should be edhec email)
    password = '###', # Input your snowflake password
    region = 'eu-central-1',
    database = 'GROUP_DB_7',
    schema = 'GROUP_DB_7.DATA_SOURCE', # Change schema to pull other tables
    warehouse= 'COMPUTE_WH',
    role = 'GROUP_ROLE_7',
)
engine = create_engine(url)
connection = engine.connect()

In [ ]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()
inspector.get_table_names(schema='HACKATHON')

/usr/local/lib/python3.7/dist-packages/snowflake/sqlalchemy/snowdialect.py:590: SAWarning: Dialect snowflake:snowflake will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  "SHOW /* sqlalchemy:get_schema_names */ SCHEMAS"))


['fe01_base_dataset_test_v1',
 'fe01_base_dataset_train_v1',
 'fe02_base_dataset_test',
 'fe02_base_dataset_train',
 'fe03_base_dataset_test',
 'fe03_base_dataset_train',
 'fe04_base_dataset_test_without_prev_sales',
 'fe04_base_dataset_test_with_prev_sales',
 'fe04_base_dataset_train_without_prev_sales',
 'fe04_base_dataset_train_with_prev_sales',
 'fe05_base_dataset_test_without_prev_sales',
 'fe05_base_dataset_test_with_prev_sales',
 'fe05_base_dataset_train_without_prev_sales',
 'fe05_base_dataset_train_with_prev_sales',
 'la01_test_dataset',
 'la01_unmapped_id_prod_season_region',
 'la02_fe01_base_dataset_test_v1',
 'la02_test_dataset_prod_feat',
 'la02_unmapped_id_prod_season_region',
 'la03_test_dataset_prod_sales_feat',
 'la04_test_dataset_prod_sales_catsales_feat',
 'la056_test_dataset_with_prev_sales',
 'la05_test_dataset_with_all_possible_features',
 'la06_test_dataset_without_prev_sales',
 'la06_test_dataset_with_prev_sales',
 'p01_iditem_color',
 'p01_iditem_test_sales',
 

## Loading the train datasets

In [ ]:
# Loading train dataset for all products
query_1 = '''
SELECT * FROM GROUP_DB_7.HACKATHON.FE05_BASE_DATASET_TRAIN_WITH_PREV_SALES

'''
df_train = pd.read_sql(query_1, connection, index_col='id_prod_season_region')
print(df_train.shape)
df_train.head(5)

(244186, 39)


,brand_name,idproduct,region,sales_season,sales_season_numeric,total_sales,product_in_train,number_of_sizes,shelf,department,...,carelabel_int,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales,prev2_subfamily_sales,prev3_subfamily_sales,prev4_subfamily_sales
id_prod_season_region,,,,,,,,,,,,,,,,,,,,,
FRANCE-E19-2-00016494,MAJE,2-00016494,FRANCE,E19,5,83.0,1,1,ACCESSOIRES,MAROQUINERIE,...,0,12943.0,3743.0,6669.0,10200.0,11934.0,12053.0,548.0,438.0,5613.0
NA-E19-2-00016753,MAJE,2-00016753,NA,E19,5,6.0,1,1,ACCESSOIRES,BIJOUX,...,0,1.0,164775.0,16017.0,74.0,2102.0,1076.0,55.0,2570.0,755.0
EUR-H20-2-00017257,MAJE,2-00017257,EUR,H20,8,39.0,1,4,WOMEN,BAS,...,0,6328.0,272885.0,8741.0,22729.0,16603.0,20839.0,18080.0,637.0,2106.0
FRANCE-H20-3-00011653,CLAUDIE,3-00011653,FRANCE,H20,8,131.0,1,4,WOMEN,DESSUS,...,1,4170.0,225841.0,11404.0,11.0,12838.0,2761.0,17.0,409.0,96.0
NA-E19-1-00031140,SANDRO,1-00031140,NA,E19,5,692.0,1,7,MEN,HAUTS,...,1,215.0,51711.0,191960.0,727.0,1584.0,937.0,1.0,400.0,146.0


In [ ]:
# Loading train dataset for products without (fresh) previous sales
query_1 = '''
SELECT * FROM GROUP_DB_7.HACKATHON.FE05_BASE_DATASET_TRAIN_WITHOUT_PREV_SALES

'''
df_train_2 = pd.read_sql(query_1, connection, index_col='id_prod_season_region')
print(df_train_2.shape)
df_train_2.head(5)

(244186, 35)


,brand_name,idproduct,region,sales_season,sales_season_numeric,total_sales,product_in_train,number_of_sizes,shelf,department,...,carelabel_int,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales,prev2_subfamily_sales,prev3_subfamily_sales,prev4_subfamily_sales
id_prod_season_region,,,,,,,,,,,,,,,,,,,,,
NA-E20-2-00015072,MAJE,2-00015072,NA,E20,7,2.0,1,3,WOMEN,DESSUS,...,0,-66.0,674.0,50.0,2787.0,380.0,1918.0,8566.0,985.0,5777.0
NA-E17-1-00019614,SANDRO,1-00019614,NA,E17,1,32.0,1,3,WOMEN,HAUTS,...,0,-100.0,-100.0,-100.0,12023.0,2382.0,3721.0,-100.0,-100.0,-100.0
NA-E20-1-00031140,SANDRO,1-00031140,NA,E20,7,202.0,1,7,MEN,HAUTS,...,1,130.0,65165.0,50919.0,2243.0,63.0,451.0,669.0,445.0,1712.0
FRANCE-E19-1-00031227,SANDRO,1-00031227,FRANCE,E19,5,241.0,1,1,WOMEN,ACCESSOIRES,...,1,156433.0,1820.0,371275.0,6092.0,19268.0,72.0,9864.0,80.0,18.0
EUR-E19-1-00023669,SANDRO,1-00023669,EUR,E19,5,21.0,1,3,WOMEN,HAUTS,...,0,447601.0,104886.0,5126.0,5797.0,36580.0,12243.0,849.0,12111.0,1172.0


## Loading the test datasets

In [ ]:
# Loading test dataset for products with (repeat) previous sales
query = '''
SELECT * FROM GROUP_DB_7.HACKATHON.LA06_TEST_DATASET_WITH_PREV_SALES
'''
df_test_1 = pd.read_sql(query, connection,index_col='id_prod_season_region')
print(df_test_1.shape)
df_test_1.head(5)

(48720, 38)


,brand_name,idproduct,region,sales_season,sales_season_numeric,total_sales,product_in_train,number_of_sizes,shelf,department,...,family_treated,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales,prev2_subfamily_sales,prev3_subfamily_sales,prev4_subfamily_sales
id_prod_season_region,,,,,,,,,,,,,,,,,,,,,
EUR-H21-1-00032829,SANDRO,1-00032829,EUR,H21,10,73.0,1,5,WOMEN,ROBES,...,ROBE,514.0,389626.0,115266.0,16214.0,27617.0,6771.0,-100.0,-100.0,-100.0
FRANCE-H21-3-00009634,CLAUDIE,3-00009634,FRANCE,H21,10,1.0,1,3,WOMEN,HAUTS,...,TOP,237498.0,796.0,265897.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
ASIA-E21-3-00008178,CLAUDIE,3-00008178,ASIA,E21,9,4.0,1,3,WOMEN,HAUTS,...,TOP,28328.0,965.0,21602.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
FRANCE-E21-1-00033792,SANDRO,1-00033792,FRANCE,E21,9,448.0,1,5,WOMEN,HAUTS,...,CARDIGAN,161957.0,499.0,583.0,20413.0,16139.0,18208.0,15.0,3636.0,44.0
EUR-H21-3-00011342,CLAUDIE,3-00011342,EUR,H21,10,39.0,1,5,WOMEN,ROBES,...,ROBE,133873.0,114017.0,3813.0,3386.0,2802.0,5789.0,-100.0,-100.0,-100.0


In [ ]:
# Loading test dataset for products without (fresh) previous sales
query = '''
SELECT * FROM GROUP_DB_7.HACKATHON.LA06_TEST_DATASET_WITHOUT_PREV_SALES
'''
df_test_2 = pd.read_sql(query, connection,index_col='id_prod_season_region')
print(df_test_2.shape)
df_test_2.head(5)

(16264, 34)


,brand_name,idproduct,region,sales_season,sales_season_numeric,total_sales,product_in_train,number_of_sizes,shelf,department,...,family_treated,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales,prev2_subfamily_sales,prev3_subfamily_sales,prev4_subfamily_sales
id_prod_season_region,,,,,,,,,,,,,,,,,,,,,
NA-H21-1-00034695,SANDRO,1-00034695,NA,H21,10,110.0,0,6.0,MEN,DESSUS,...,MANTEAU,111.0,47.0,60440.0,850.0,11238.0,923.0,201.0,64.0,57.0
FRANCE-H21-3-00013186,CLAUDIE,3-00013186,FRANCE,H21,10,1.0,0,4.0,WOMEN,ROBES,...,ROBE,237498.0,796.0,265897.0,33107.0,10343.0,6738.0,-100.0,-100.0,-100.0
ASIA-E21-2-00016554,MAJE,2-00016554,ASIA,E21,9,2.0,0,3.0,WOMEN,DESSUS,...,MANTEAU,9423.0,145.0,4440.0,7213.0,18070.0,3098.0,481.0,352.0,275.0
FRANCE-E21-1-00023227,SANDRO,1-00023227,FRANCE,E21,9,2.0,0,4.0,WOMEN,DESSUS,...,MANTEAU,161957.0,499.0,583.0,9637.0,58822.0,2262.0,236.0,104.0,3.0
ASIA-E21-2-00017212,MAJE,2-00017212,ASIA,E21,9,2.0,0,3.0,WOMEN,HAUTS,...,ROBE,9423.0,145.0,4440.0,18070.0,3098.0,13213.0,-100.0,-100.0,-100.0


In [ ]:
# Loading combined test dataset for both fresh and repeat
query = '''
SELECT * FROM GROUP_DB_7.HACKATHON.LA05_TEST_DATASET_WITH_ALL_POSSIBLE_FEATURES
'''
df_test = pd.read_sql(query, connection,index_col='id_prod_season_region')
print(df_test.shape)
pd.set_option('display.max_columns', None)
df_test.head(5)

(64984, 38)


,brand_name,idproduct,region,sales_season,sales_season_numeric,total_sales,product_in_train,number_of_sizes,shelf,department,subfamily,carelabel,carelabel_int,storagetype,valfrsupplytype,valfrfashionlevel,valfrpattern,valfrpricesegmentation,main_elt_treated,prev1_product_sales,prev2_product_sales,prev3_product_sales,prev4_product_sales,wash_difficulty,bleach_difficulty,dry_difficulty,iron_difficulty,dryclean_difficulty,family_treated,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales,prev2_subfamily_sales,prev3_subfamily_sales,prev4_subfamily_sales
id_prod_season_region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ASIA-E21-1-00026814,SANDRO,1-00026814,ASIA,E21,9,1.0,1,6.0,MEN,COSTUMES & SMOCKINGS,PANTALON,False,0.0,SUSPENDU,NEGOCE,None,UNI,ENTREE GAMME,LAINE,5.0,18.0,35.0,18.0,4,0,2,1,2,PANTS,9978.0,42257.0,341280.0,250.0,582.0,7084.0,3606.0,156.0,250.0
NA-E21-1-00034539,SANDRO,1-00034539,NA,E21,9,24.0,1,5.0,WOMEN,DESSUS,VESTE TAILLEUR,True,1.0,SUSPENDU,FACON,None,IMPRIMÉ,None,VISCOSE,-100.0,-100.0,-100.0,-100.0,4,0,2,1,2,VESTE,109.0,47344.0,117031.0,30602.0,1746.0,3267.0,1.0,-1.0,3.0
FRANCE-H21-1-00028259,SANDRO,1-00028259,FRANCE,H21,10,3.0,1,7.0,MEN,HAUTS,MANCHES LONGUES,True,1.0,CARTON,NEGOCE,None,UNI,ENTREE GAMME,COTON,3.0,20.0,25.0,116.0,2,0,2,1,1,SWEATER,151331.0,375279.0,380853.0,9679.0,17789.0,19775.0,89.0,9839.0,209.0
ASIA-E21-1-00034389,SANDRO,1-00034389,ASIA,E21,9,760.0,1,5.0,WOMEN,HAUTS,CHEMISE,True,1.0,SUSPENDU,FACON,None,IMPRIMÉ,None,SOIE,-100.0,-100.0,-100.0,-100.0,4,0,2,1,2,SHIRT,42257.0,341280.0,6074.0,5900.0,8785.0,58113.0,932.0,123.0,219.0
FRANCE-H21-1-00034766,SANDRO,1-00034766,FRANCE,H21,10,825.0,1,5.0,WOMEN,BAS,COMBI-PANTALON,True,1.0,SUSPENDU,FACON,None,IMPRIMÉ,None,COTON,824.0,-100.0,-100.0,-100.0,4,0,2,1,2,OTHERS,141538.0,407.0,151331.0,7362.0,9135.0,46837.0,63.0,120.0,249.0


# Preprocessing

## Existing Products

### Train Dataset

In [ ]:
# Removing the rows that do not have previous product sales (i.e we keep only products that have previous sales)
df_train_1 = df_train[df_train.prev1_product_sales != -100]
df_train_1 = df_train[df_train.prev2_product_sales != -100]

In [ ]:
# Drop the target and variables shown to be unnecessary
labels = ['prev2_subfamily_sales', 'sales_season_numeric',	'prev4_product_sales', 'subfamily', 'wash_difficulty',	'bleach_difficulty',	'dry_difficulty',	'iron_difficulty',	'dryclean_difficulty', 'carelabel_int', 'shelf', 'idproduct', 'storagetype', 'valfrsupplytype', 'valfrpattern', 'family_treated', 'total_sales' , 'sales_season', 'product_in_train', 'family', 'department', 'sec_elt', 'valfrpricesegmentation', 'main_elt_treated', 'prev3_subfamily_sales', 'prev4_subfamily_sales']

# Loading the X set
X_train_1 = df_train_1.copy()
X_train_1.drop(labels = labels, axis=1, inplace=True)

# Loading the y set
y_train_1 = df_train_1['total_sales'].copy()

print(X_train_1.shape)
X_train_1.head(10)

(116570, 13)


,brand_name,region,number_of_sizes,carelabel,prev1_product_sales,prev2_product_sales,prev3_product_sales,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales
id_prod_season_region,,,,,,,,,,,,,
FRANCE-E19-2-00016494,MAJE,FRANCE,1,False,196.0,1.0,-100.0,12943.0,3743.0,6669.0,10200.0,11934.0,12053.0
EUR-H20-2-00017257,MAJE,EUR,4,False,71.0,160.0,303.0,6328.0,272885.0,8741.0,22729.0,16603.0,20839.0
NA-H19-1-00029668,SANDRO,NA,8,True,39.0,168.0,-100.0,200886.0,215.0,51711.0,620.0,13719.0,8962.0
EUR-H20-3-00010360,CLAUDIE,EUR,3,True,94.0,409.0,15.0,95152.0,2262.0,163635.0,2196.0,635.0,11787.0
ASIA-H18-1-00026317,SANDRO,ASIA,4,False,25.0,96.0,-100.0,12478.0,264877.0,210978.0,2399.0,52.0,18699.0
NA-H18-1-00023134,SANDRO,NA,4,False,58.0,24.0,127.0,43300.0,182702.0,296.0,2926.0,48149.0,7998.0
NA-H20-1-00023134,SANDRO,NA,4,False,1.0,18.0,58.0,47344.0,117031.0,136.0,31594.0,7966.0,4928.0
EUR-E20-3-00010360,CLAUDIE,EUR,3,True,409.0,15.0,-100.0,163635.0,5224.0,3223.0,6232.0,6786.0,8569.0
EUR-E20-3-00009286,CLAUDIE,EUR,4,False,44.0,32.0,374.0,163635.0,5224.0,3223.0,2.0,3141.0,1439.0


In [ ]:
# Check for nulls
X_train_1.isna().sum().sum()

0

### Test Dataset

In [ ]:
labels = ['prev2_subfamily_sales', 'sales_season_numeric',	'prev4_product_sales','bleach_difficulty',	'dry_difficulty',	'iron_difficulty', 'wash_difficulty',	'dryclean_difficulty', 'valfrfashionlevel', 'department', 'main_elt_treated', 'carelabel_int', 'shelf', 'idproduct', 'storagetype', 'valfrsupplytype', 'valfrpattern', 'family_treated', 'total_sales', 'subfamily', 'sales_season', 'product_in_train', 'valfrpricesegmentation', 'prev3_subfamily_sales', 'prev4_subfamily_sales']

# Loading the X set
X_test_1 = df_test_1.copy()
X_test_1.drop(labels = labels, axis=1, inplace=True)

# Loading the y set
y_test_1 = df_test_1['total_sales'].copy()

print(X_test_1.shape)
X_test_1.head(5)

(48720, 13)


,brand_name,region,number_of_sizes,carelabel,prev1_product_sales,prev2_product_sales,prev3_product_sales,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales
id_prod_season_region,,,,,,,,,,,,,
EUR-H21-1-00032829,SANDRO,EUR,5,True,71.0,731.0,1136.0,514.0,389626.0,115266.0,16214.0,27617.0,6771.0
FRANCE-H21-3-00009634,CLAUDIE,FRANCE,3,True,1.0,17.0,36.0,237498.0,796.0,265897.0,-100.0,-100.0,-100.0
ASIA-E21-3-00008178,CLAUDIE,ASIA,3,False,7.0,8.0,1.0,28328.0,965.0,21602.0,-100.0,-100.0,-100.0
FRANCE-E21-1-00033792,SANDRO,FRANCE,5,True,374.0,-100.0,-100.0,161957.0,499.0,583.0,20413.0,16139.0,18208.0
EUR-H21-3-00011342,CLAUDIE,EUR,5,True,39.0,131.0,338.0,133873.0,114017.0,3813.0,3386.0,2802.0,5789.0


In [ ]:
# Categorical data
categorical_cols = ['brand_name', 'region', 'carelabel']

# Get dummies for categorical data
X_train_1_final = pd.get_dummies(X_train_1, columns = categorical_cols)

X_test_1_final = pd.get_dummies(X_test_1, columns = categorical_cols)

In [ ]:
# Check for untrained levels and/or missing levels
missing_levels = X_train_1_final.columns.difference(X_test_1_final.columns)
pd.Index([u'D'], dtype='object')

untrained_levels = X_test_1_final.columns.difference(X_train_1_final.columns)
pd.Index([u'D'], dtype='object')

Index(['D'], dtype='object')

In [ ]:
# Drop any levels from the train set that are not in the test set
X_train_1_final.drop(labels = missing_levels, axis=1, inplace=True)

X_test_1_final.drop(labels = untrained_levels, axis=1, inplace=True)

In [ ]:
# Print shapes of train and test data for modelling
print(X_train_1_final.shape, X_test_1_final.shape)

(116570, 19) (48720, 19)


## New products

### Train dataset

In [ ]:
# Removing the rows that have previous product sales (i.e we keep only products that are new)
df_train_2 = df_train[df_train.prev1_product_sales == -100]
df_train_2 = df_train[df_train.prev2_product_sales == -100]

In [ ]:
# Drop the target and variables shown to be unnecessary
labels = ['prev2_subfamily_sales', 'sales_season_numeric', 'prev1_product_sales',	'prev2_product_sales',	'prev3_product_sales',	'prev4_product_sales', 'subfamily', 'wash_difficulty',	'bleach_difficulty',	'dry_difficulty',	'iron_difficulty',	'dryclean_difficulty', 'carelabel_int', 'shelf', 'idproduct', 'storagetype', 'valfrsupplytype', 'valfrpattern', 'family_treated', 'total_sales' , 'sales_season', 'product_in_train', 'family', 'department', 'sec_elt', 'valfrpricesegmentation', 'main_elt_treated', 'prev3_subfamily_sales', 'prev4_subfamily_sales']

# Loading the X set
X_train_2 = df_train_2.copy()
X_train_2.drop(labels = labels, axis=1, inplace=True)

# Loading the y set
y_train_2 = df_train_2['total_sales'].copy()

print(X_train_2.shape)
X_train_2.head(10)

(127616, 10)


,brand_name,region,number_of_sizes,carelabel,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales
id_prod_season_region,,,,,,,,,,
NA-E19-2-00016753,MAJE,NA,1,False,1.0,164775.0,16017.0,74.0,2102.0,1076.0
FRANCE-H20-3-00011653,CLAUDIE,FRANCE,4,True,4170.0,225841.0,11404.0,11.0,12838.0,2761.0
NA-E19-1-00031140,SANDRO,NA,7,True,215.0,51711.0,191960.0,727.0,1584.0,937.0
EUR-E19-1-00031900,SANDRO,EUR,6,False,498329.0,1932.0,447601.0,1730.0,408.0,2449.0
EUR-E20-2-00019675,MAJE,EUR,3,True,8399.0,13523.0,431722.0,7128.0,22292.0,5901.0
FRANCE-E17-1-00020813,SANDRO,FRANCE,5,False,441268.0,-100.0,-100.0,34660.0,30716.0,3179.0
FRANCE-H17-1-00020813,SANDRO,FRANCE,5,False,166384.0,4598.0,441268.0,25488.0,19030.0,3923.0
EUR-E19-3-00010360,CLAUDIE,EUR,3,True,160950.0,149602.0,2485.0,2409.0,2662.0,6899.0
EUR-H19-2-00017700,MAJE,EUR,3,True,13786.0,386281.0,1889.0,11790.0,39373.0,7436.0


In [ ]:
# Check for nulls
X_train_2.isna().sum().sum()

0

### Test dataset

In [ ]:
labels = ['prev2_subfamily_sales', 'sales_season_numeric','bleach_difficulty',	'dry_difficulty',	'iron_difficulty', 'wash_difficulty',	'dryclean_difficulty', 'valfrfashionlevel', 'department', 'main_elt_treated', 'carelabel_int', 'shelf', 'idproduct', 'storagetype', 'valfrsupplytype', 'valfrpattern', 'family_treated', 'total_sales', 'subfamily', 'sales_season', 'product_in_train', 'valfrpricesegmentation', 'prev3_subfamily_sales', 'prev4_subfamily_sales']

# Loading the X set
X_test_2 = df_test_2.copy()
X_test_2.drop(labels = labels, axis=1, inplace=True)

# Loading the y set
y_test_2 = df_test_2['total_sales'].copy()

print(X_test_2.shape)
X_test_2.head(5)

(16264, 10)


,brand_name,region,number_of_sizes,carelabel,prev2_shelf_sales,prev3_shelf_sales,prev4_shelf_sales,prev2_family_sales,prev3_family_sales,prev4_family_sales
id_prod_season_region,,,,,,,,,,
NA-H21-1-00034695,SANDRO,NA,6.0,True,111.0,47.0,60440.0,850.0,11238.0,923.0
FRANCE-H21-3-00013186,CLAUDIE,FRANCE,4.0,True,237498.0,796.0,265897.0,33107.0,10343.0,6738.0
ASIA-E21-2-00016554,MAJE,ASIA,3.0,False,9423.0,145.0,4440.0,7213.0,18070.0,3098.0
FRANCE-E21-1-00023227,SANDRO,FRANCE,4.0,False,161957.0,499.0,583.0,9637.0,58822.0,2262.0
ASIA-E21-2-00017212,MAJE,ASIA,3.0,True,9423.0,145.0,4440.0,18070.0,3098.0,13213.0


In [ ]:
# Check for null values
X_test_2.isna().sum()

brand_name            0
region                0
number_of_sizes       1
carelabel             5
prev2_shelf_sales     0
prev3_shelf_sales     0
prev4_shelf_sales     0
prev2_family_sales    0
prev3_family_sales    0
prev4_family_sales    0
dtype: int64

In [ ]:
# Impute null values with mode
X_test_2['carelabel'] = X_test_2['carelabel'].fillna(X_test_2['carelabel'].mode()[0])
X_test_2['number_of_sizes'] = X_test_2['number_of_sizes'].fillna(X_test_2['number_of_sizes'].mode()[0])

In [ ]:
# Categorical data
categorical_cols = ['brand_name', 'region', 'carelabel']

# Get dummies for categorical data
X_train_2_final = pd.get_dummies(X_train_2, columns = categorical_cols)

X_test_2_final = pd.get_dummies(X_test_2, columns = categorical_cols)

In [ ]:
# Check for untrained levels and/or missing levels
missing_levels = X_train_2_final.columns.difference(X_test_2_final.columns)
pd.Index([u'D'], dtype='object')

untrained_levels = X_test_2_final.columns.difference(X_train_2_final.columns)
pd.Index([u'D'], dtype='object')

Index(['D'], dtype='object')

In [ ]:
# Drop any levels from the train set that are not in the test set
X_train_2_final.drop(labels = missing_levels, axis=1, inplace=True)

X_test_2_final.drop(labels = untrained_levels, axis=1, inplace=True)

In [ ]:
# Print the shapes of train and test data for modelling
print(X_train_2_final.shape, X_test_2_final.shape)

(127616, 16) (16264, 16)


# Modelling

## Existing Products

### Dummy (baseline) model

We will fit a dummy model to act as our baseline with Mean Absolute Error as our metric of choice

In [ ]:
# Dummy model that predicts the mean
dummy = dr()
dummy.fit(X_test_1, y_test_1)

# Check score
dummy_acc = metrics.mean_absolute_error(y_test_1, dummy.predict(X_test_1))
print('MAE of dummy for existing products: %.5s' % dummy_acc)

MAE of dummy for existing products: 125.8


### XGBoost

In [ ]:
# Prepare data for xgb
data_dmatrix = xgb.DMatrix(data=X_train_1_final,label=y_train_1)

# Initialise xgb
xg_reg = xgb.XGBRegressor(booster='dart', colsample_bylevel=1, colsample_bynode=1, colsample_bytree = 0.8, learning_rate = 0.05,
                max_depth = 3, alpha = 0.1, min_child_weight=10, n_estimators = 15, missing=0, verbosity=0, col_sample_rate=0.6, col_sample_rate_per_tree=0.7,
                distribution='gaussian', fold_assignment=None, min_rows=10, score_tree_interval=5)

# Fit the model on train set
xg_reg.fit(X_train_1_final, y_train_1)

# Make predictions for test set
y_pred_xgb = xg_reg.predict(X_test_1_final)

In [ ]:
xgb_acc = metrics.mean_absolute_error(y_test_1, y_pred_xgb)
print('MAE of XGB for existing products: %s' % xgb_acc)

MAE of XGB for existing products: 93.17944496965957


### Random Forest

In [ ]:
# Load and fit random forest model
rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train_1_final, y_train_1)

# Make predictions for test set
y_pred_rf = rf.predict(X_test_1_final)

In [ ]:
# Make predictions
rf_acc = metrics.mean_absolute_error(y_test_1, y_pred_rf)
print('MAE of Random Forest for existing products: %s' % rf_acc)

MAE of Random Forest for existing products: 76.56734960700275


In [ ]:
# Show predicted and true values
y_preds_rf = pd.DataFrame(data=y_pred_rf, columns=['predicted'], index=X_test_1_final.index)
results_rf = y_preds_rf.merge(y_test_1, how='inner', left_index=True, right_index=True)
print("Random Forest model predictions for existing products")
results_rf

Random Forest model predictions for existing products


,predicted,total_sales
id_prod_season_region,,
EUR-H21-1-00032829,107.37,73.0
FRANCE-H21-3-00009634,23.51,1.0
ASIA-E21-3-00008178,3.36,4.0
FRANCE-E21-1-00033792,152.65,448.0
EUR-H21-3-00011342,24.79,39.0
...,...,...
EUR-H21-2-00020966,127.10,390.0
ASIA-H21-1-00034444,40.15,39.0
FRANCE-H21-2-00018531,48.72,43.0


### Decision Tree

In [ ]:
# Load and fit decision tree regressor
dt = DecisionTreeRegressor()
dt.fit(X_train_1_final, y_train_1)

# Make predictions for test set
y_pred_dt = dt.predict(X_test_1_final)

In [ ]:
# Make predictions
dt_acc = metrics.mean_absolute_error(y_test_1, y_pred_dt)
print('MAE of Decision Tree for existing products: %s' % dt_acc)

MAE of Decision Tree for existing products: 98.45932197591681


### Stacked models

In [ ]:
# define the base models
level0 = list()
level0.append(('xgb', xgb.XGBRegressor(booster='dart', colsample_bylevel=0.6, colsample_bynode=1, colsample_bytree = 0.8, learning_rate = 0.05,
                max_depth = 15, alpha = 10, min_child_weight=3, n_estimators = 10, missing=-100, verbosity=0, fold_assignment="Modulo", keep_cross_validation_predictions = True)))
level0.append(('rf', RandomForestRegressor(n_estimators = 100)))
level0.append(('lgb', LGBMRegressor(fold_assignment="Modulo", keep_cross_validation_predictions = True))) 
level0.append(('lr', LinearRegression())) 
level0.append(('dt', DecisionTreeRegressor()))


# define meta learner model
level1 = xgb.XGBRegressor()

# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

In [ ]:
# fit the model on all available data
model.fit(X_train_1_final, y_train_1)

[00:40:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(cv=5,
                  estimators=[('xgb',
                               XGBRegressor(alpha=10, booster='dart',
                                            colsample_bylevel=0.6,
                                            colsample_bytree=0.8,
                                            fold_assignment='Modulo',
                                            keep_cross_validation_predictions=True,
                                            learning_rate=0.05, max_depth=15,
                                            min_child_weight=3, missing=-100,
                                            n_estimators=10, verbosity=0)),
                              ('rf', RandomForestRegressor()),
                              ('lgb',
                               LGBMRegressor(fold_assignment='Modulo',
                                             keep_cross_validation_predictions=True)),
                              ('lr', LinearRegression()),
                              ('

In [ ]:
# make a prediction for one example
y_pred_stacked = model.predict(X_test_1_final)

# Print accuracy
stacked_acc = metrics.mean_absolute_error(y_test_1, y_pred_stacked)
print('MAE of Stacked model for existing products: %s' % stacked_acc)

MAE of Stacked model for existing products: 75.52541816620209


In [ ]:
# Show predicted and true values
y_preds_stacked = pd.DataFrame(data=y_pred_stacked, columns=['pred'], index=X_test_1_final.index)
results_stacked = y_preds_stacked.merge(y_test_1, how='inner', left_index=True, right_index=True)
print("Stacked model predictions for existing products")
results_stacked

Stacked model predictions for existing products


,pred,total_sales
id_prod_season_region,,
EUR-H21-1-00032829,126.565445,73.0
FRANCE-H21-3-00009634,19.664459,1.0
ASIA-E21-3-00008178,7.422267,4.0
FRANCE-E21-1-00033792,167.707153,448.0
EUR-H21-3-00011342,25.378044,39.0
...,...,...
EUR-H21-2-00020966,114.945930,390.0
ASIA-H21-1-00034444,39.430183,39.0
FRANCE-H21-2-00018531,53.111469,43.0


## Fresh Products

In [ ]:
# Dummy model that predicts the mean
dummy = dr()
dummy.fit(X_test_2, y_test_2)

# Check score
dummy_acc_2 = metrics.mean_absolute_error(y_test_2, dummy.predict(X_test_2))
print('MAE of dummy for new products: %.5s' % dummy_acc_2)

MAE of dummy for new products: 101.2


### XGBoost

In [ ]:
# Prepare data for xgb
data_dmatrix = xgb.DMatrix(data=X_train_2_final,label=y_train_2)

# Initialise xgb
xg_reg_2 = xgb.XGBRegressor(booster='dart', colsample_bylevel=1, colsample_bynode=1, colsample_bytree = 0.8, learning_rate = 0.05,
                max_depth = 3, alpha = 0.1, min_child_weight=10, n_estimators = 15, missing=0, verbosity=1, col_sample_rate=0.6, col_sample_rate_per_tree=0.7,
                distribution='gaussian', fold_assignment=None, min_rows=10, score_tree_interval=5)

# Fit the model on train set
xg_reg_2.fit(X_train_2_final, y_train_2)

# Make predictions for test set
y_pred_xgb_2 = xg_reg_2.predict(X_test_2_final)

[00:40:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
xgb_acc_2 = metrics.mean_absolute_error(y_test_2, y_pred_xgb_2)
print('MAE of XGB: %s' % xgb_acc)

MAE of XGB: 93.17944496965957


### Random Forest

In [ ]:
# Load and fit random forest model
rf_2 = RandomForestRegressor(n_estimators = 100)
rf_2.fit(X_train_2_final, y_train_2)

# Make predictions for test set
y_pred_rf_2 = rf_2.predict(X_test_2_final)

In [ ]:
# Make predictions
rf_acc_2 = metrics.mean_absolute_error(y_test_2, y_pred_rf_2)
print('MAE of Random Forest for new products: %s' % rf_acc_2)

MAE of Random Forest for new products: 204.9375019806279


### Decision Trees

In [ ]:
# Load and fit decision tree regressor
dt_2 = DecisionTreeRegressor()
dt_2.fit(X_train_2_final, y_train_2)

# Make predictions for test set
y_pred_dt_2 = dt_2.predict(X_test_2_final)

### Stacked model

In [ ]:
# define the base models
level0 = list()
level0.append(('xgb', xgb.XGBRegressor(booster='dart', colsample_bylevel=0.6, colsample_bynode=1, colsample_bytree = 0.8, learning_rate = 0.05,
                max_depth = 15, alpha = 10, min_child_weight=3, n_estimators = 10, missing=-100, verbosity=0, fold_assignment="Modulo", keep_cross_validation_predictions = True)))
#level0.append(('rf', RandomForestRegressor(n_estimators = 100)))
level0.append(('lgb', LGBMRegressor(fold_assignment="Modulo", keep_cross_validation_predictions = True))) 
level0.append(('lr', LinearRegression())) 

# define meta learner model
level1 = xgb.XGBRegressor()

# define the stacking ensemble
model_2 = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

In [ ]:
# fit the model on all available data
model_2.fit(X_train_2_final, y_train_2)

[00:41:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


StackingRegressor(cv=5,
                  estimators=[('xgb',
                               XGBRegressor(alpha=10, booster='dart',
                                            colsample_bylevel=0.6,
                                            colsample_bytree=0.8,
                                            fold_assignment='Modulo',
                                            keep_cross_validation_predictions=True,
                                            learning_rate=0.05, max_depth=15,
                                            min_child_weight=3, missing=-100,
                                            n_estimators=10, verbosity=0)),
                              ('lgb',
                               LGBMRegressor(fold_assignment='Modulo',
                                             keep_cross_validation_predictions=True)),
                              ('lr', LinearRegression())],
                  final_estimator=XGBRegressor())

In [ ]:
# make a prediction for one example
y_pred_stacked_2 = model_2.predict(X_test_2_final)

# Print accuracy
stacked_acc_2 = metrics.mean_absolute_error(y_test_2, y_pred_stacked_2)
print('MAE of Stacked model for new products: %s' % stacked_acc_2)

MAE of Stacked model for new products: 184.8796645024699


### Deep Learning

In [ ]:
# Load the deep learning regression model
dl = Sequential()
dl.add(Dense(50, input_dim=X_train_2_final.shape[1], kernel_initializer='normal', activation='relu'))
dl.add(Dense(25, input_dim=X_train_2_final.shape[1], kernel_initializer='normal', activation='relu'))
dl.add(Dense(1, kernel_initializer='normal'))
dl.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])

def baseline_model():
  dl = Sequential()
  dl.add(Dense(50, input_dim=X_train_2_final.shape[1], kernel_initializer='normal', activation='relu'))
  dl.add(Dense(25, input_dim=X_train_2_final.shape[1], kernel_initializer='normal', activation='relu'))
  dl.add(Dense(1, kernel_initializer='normal'))

  # Compile the model
  dl.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
  return dl

In [ ]:
# Set a checkpoint
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_mae', verbose=1,
    save_best_only=True, mode='auto', period=1, save_weights_only=True)

# Fit the model
estimator = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=100, verbose=0, callbacks=[checkpoint])
estimator.fit(X_train_2_final, y_train_2, validation_data=(X_test_2_final, y_test_2))

# Save the best weights/model
files.download("/content/best_model.hdf5" ) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  



Epoch 1: val_mae improved from inf to 81.19868, saving model to best_model.hdf5

Epoch 2: val_mae did not improve from 81.19868

Epoch 3: val_mae did not improve from 81.19868

Epoch 4: val_mae did not improve from 81.19868

Epoch 5: val_mae did not improve from 81.19868

Epoch 6: val_mae did not improve from 81.19868

Epoch 7: val_mae did not improve from 81.19868

Epoch 8: val_mae did not improve from 81.19868

Epoch 9: val_mae did not improve from 81.19868

Epoch 10: val_mae did not improve from 81.19868

Epoch 11: val_mae did not improve from 81.19868

Epoch 12: val_mae did not improve from 81.19868

Epoch 13: val_mae did not improve from 81.19868

Epoch 14: val_mae did not improve from 81.19868

Epoch 15: val_mae did not improve from 81.19868

Epoch 16: val_mae did not improve from 81.19868

Epoch 17: val_mae did not improve from 81.19868

Epoch 18: val_mae did not improve from 81.19868

Epoch 19: val_mae did not improve from 81.19868

Epoch 20: val_mae did not improve from 81.19

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the best model
dl.load_weights("/content/best_model.hdf5")

# Make predictions
y_pred_dl = dl.predict(X_test_2_final)

# Print accuracy
dl_acc = metrics.mean_absolute_error(y_test_2, y_pred_dl)
print('MAE of deep learning model for new products: %s' % dl_acc)

MAE of deep learning model for new products: 81.19867504673218


In [ ]:
# Show predicted and true values
y_preds_dl = pd.DataFrame(data=y_pred_dl, columns=['predicted'], index=X_test_2_final.index)
results_dl = y_preds_dl.merge(y_test_2, how='inner', left_index=True, right_index=True)
print("Deep Learning model predictions for new products")
results_dl

Deep Learning model predictions for new products


,predicted,total_sales
id_prod_season_region,,
NA-H21-1-00034695,12.229631,110.0
FRANCE-H21-3-00013186,51.857506,1.0
ASIA-E21-2-00016554,17.251429,2.0
FRANCE-E21-1-00023227,49.531921,2.0
ASIA-E21-2-00017212,30.297295,2.0
...,...,...
FRANCE-H21-3-00008902,32.628601,1.0
EUR-E21-2-00017588,31.909916,12.0
ASIA-H21-3-00012657,5.451611,86.0


# Results

Showing the results of models for existing and new products

In [ ]:
# Showing the results table (metric - Mean Absoulte Error)
index_i = pd.Series(['Dummy', 'XGBoost', 'Random Forest', 'Stacked', 'Deep Learning'])
sets = ['Existing', 'New']
results_existing = [dummy_acc, xgb_acc,rf_acc, stacked_acc, '-']
results_new = [dummy_acc_2, xgb_acc_2,rf_acc_2,stacked_acc_2, dl_acc]

results_final = pd.DataFrame(index=index_i, columns=sets)

results_final['Existing'] = results_existing
results_final['New'] = results_new
results_final = round(results_final, 0)
results_final

,Existing,New
Dummy,125.821279,101.0
XGBoost,93.179445,113.0
Random Forest,76.56735,205.0
Stacked,75.525418,185.0
Deep Learning,-,81.0


In [ ]:
df_test_results = df_test[['brand_name',	'region',	'sales_season', 'total_sales']].copy()
predictions = y_preds_rf.round(0).append(y_preds_dl.round(0))
df_test_results = df_test_results.merge(predictions.round(0), how='left', left_index=True, right_index=True)

df_test_results.head(5)

,brand_name,region,sales_season,total_sales,predicted
id_prod_season_region,,,,,
ASIA-E21-1-00026814,SANDRO,ASIA,E21,1.0,60.0
NA-E21-1-00034539,SANDRO,NA,E21,24.0,16.0
FRANCE-H21-1-00028259,SANDRO,FRANCE,H21,3.0,29.0
ASIA-E21-1-00034389,SANDRO,ASIA,E21,760.0,40.0
FRANCE-H21-1-00034766,SANDRO,FRANCE,H21,825.0,238.0


In [ ]:
# Final MAE of given test set
mae_final = round(metrics.mean_absolute_error(df_test_results['total_sales'], df_test_results['predicted']), 2)
print("MAE of entire test set is %s" % mae_final)

MAE of entire test set is 77.72


# Miscellaneous

In [ ]:
df_test_results.to_csv("predictions_v2.csv")
files.download("predictions_v2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>